In [ ]:
import pickle
!pip install boto3
import boto3

!pip install fsspec
import fsspec

import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
#sc = spark.sparkContext         # Since Spark 2.0 'spark' is a
#SparkSession object that is by default created upfront and available
#in Spark shell, you need to explicitly create SparkSession object by
#using builder
spark = SparkSession.builder.getOrCreate()
#sc = SparkContext().getOrCreate()
sc = SparkContext._active_spark_context #devuelve la instancia existente
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField
sqlContext = SQLContext(sc,spark)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Cambiar path para correr desde otro lado**

In [ ]:
import os
base_path = '/content/drive/MyDrive/openalex-institution-parsing/'

base_save_path = "/content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/"

iteration_save_path = "/content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/"


### Getting all data (From saved OpenAlex DB snapshot)

In [ ]:
#institutions = spark.read.parquet(f"{base_save_path}/OA_static_institutions_single_file.parquet") \
 #   .filter(F.col('ror_id')!='')

#institutions_df = pd.read_parquet("/content/drive/MyDrive/openalex-institution-parsing/Archivos/OA_static_institutions_single_file.parquet")

### ALTERNATIVA DE STATIC INSTITUTIONS: BAJAR CON PYALEX PARTE DEL OBJETO INSTITUTIONS
!pip install pyalex
import pyalex
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders

from pyalex import config

config.max_retries = 0
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

pager = Institutions().filter(country_code="AR").paginate(method="page",per_page=200)

listaDeInstituciones = list()
for page in pager:
    print(len(page))
    listaDeInstituciones += page

print(len(listaDeInstituciones))

institutions_df=pd.DataFrame(listaDeInstituciones)
institutions_df.head(n=2)


institutions_df['affiliation_id'] = institutions_df['id'].apply(lambda x: x.split("/")[-1])
institutions_df['affiliation_id']=institutions_df['affiliation_id'].apply(lambda x: x.split("I")[-1])

institutions_df['ror_id'] = institutions_df['ror'].apply(lambda x: x.split("/")[-1])

##nstitutions_df['display_name'] = institutions_df['display_name_alternatives'] ##ojo lo comenté!
institutions_df.head(n=2)
print(institutions_df.shape)

200
131
0
331
(331, 31)


In [ ]:
#from pyspark.sql import Row
# spark is from the previous example.
# Create a simple DataFrame, stored into a partition directory
#sc = spark.sparkContextinstitutions.cache().count()

In [ ]:
##STATIC AFFILIATION!
##utilizando el que estaba en drive
##affiliations = spark.read.parquet(f"{'/content/drive/MyDrive/openalex-institution-parsing/Archivos'}/static_affiliations.parquet")


In [ ]:
#### ALTERNATIVA DE STATIC AFFILIATION: BAJAR DESDE OPENALEX CON PYALEX
!pip install pyalex
import pyalex
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders

#pyalex.config.email = "mfartola.mincyt@gmail.com"
from pyalex import config
#config.max_retries = 0
#config.retry_backoff_factor = 0.1
#config.retry_http_codes = [429, 500, 503]

import pandas as pd
#argentina_works=Works().filter(authorships={"institutions": {"country_code": "AR"}}).get()
#argentina_works=pd.DataFrame(argentina_works)
#argentina_works['authorships'].head()
#argentina_works.columns
#argentina_works.head()

pager = Works().filter(authorships={"institutions": {"country_code": "AR"}}).paginate(method="page",per_page=200)

listaDeWorks = list()
for page in pager:
    print(len(page))
    listaDeWorks += page

#print(len(listaDeWorks))

argentina_works=pd.DataFrame(listaDeWorks)



200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [ ]:
argentina_works['affiliations'] = argentina_works['authorships'].apply(lambda x: [i['affiliations'] for i in x] if isinstance(x,list) else [])
print(argentina_works['affiliations'].head())
print(argentina_works.shape)
argentina_works.head()


0    [[{'raw_affiliation_string': 'Center for Envir...
1    [[{'raw_affiliation_string': 'Population Healt...
2    [[{'raw_affiliation_string': 'Department of Su...
3    [[{'raw_affiliation_string': 'Department of Ec...
4    [[{'raw_affiliation_string': 'Institut Curie, ...
Name: affiliations, dtype: object
(10000, 52)


,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,referenced_works,related_works,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,fulltext_origin,is_authors_truncated,affiliations
0,https://openalex.org/W2746485780,https://doi.org/10.1038/387253a0,The value of the world's ecosystem services an...,The value of the world's ecosystem services an...,1997,1997-05-01,{'openalex': 'https://openalex.org/W2746485780...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,"[https://openalex.org/W115314074, https://open...","[https://openalex.org/W4389284537, https://ope...",None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 1163}, {'yea...",2024-11-17T19:37:30.924391,2017-08-31,NaN,NaN,[[{'raw_affiliation_string': 'Center for Envir...
1,https://openalex.org/W2136489990,https://doi.org/10.1056/nejmoa0905561,Dabigatran versus Warfarin in Patients with At...,Dabigatran versus Warfarin in Patients with At...,2009,2009-08-31,{'openalex': 'https://openalex.org/W2136489990...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,"[https://openalex.org/W1586488555, https://ope...","[https://openalex.org/W2462281387, https://ope...","{'Warfarin': [0], 'reduces': [1], 'the': [2, 1...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 311}, {'year...",2024-11-19T23:50:25.983624,2016-06-24,ngrams,NaN,[[{'raw_affiliation_string': 'Population Healt...
2,https://openalex.org/W2144173184,https://doi.org/10.1097/sla.0b013e3181b13ca2,The Clavien-Dindo Classification of Surgical C...,The Clavien-Dindo Classification of Surgical C...,2009,2009-07-31,{'openalex': 'https://openalex.org/W2144173184...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,"[https://openalex.org/W1550012216, https://ope...","[https://openalex.org/W4389568370, https://ope...","{'The': [0, 47, 65, 236], 'lack': [1], 'of': [...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 944}, {'year...",2024-11-19T22:11:30.558674,2016-06-24,ngrams,NaN,[[{'raw_affiliation_string': 'Department of Su...
3,https://openalex.org/W2098000995,https://doi.org/10.1126/science.287.5459.1770,Global Biodiversity Scenarios for the Year 2100,Global Biodiversity Scenarios for the Year 2100,2000,2000-03-10,{'openalex': 'https://openalex.org/W2098000995...,en,"{'is_oa': False, 'landing_page_url': 'https://...",review,...,"[https://openalex.org/W106885836, https://open...","[https://openalex.org/W2921401668, https://ope...","{'Scenarios': [0], 'of': [1, 16, 31, 41, 44, 4...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 365}, {'year...",2024-11-20T07:08:42.917830,2016-06-24,ngrams,NaN,[[{'raw_affiliation_string': 'Department of Ec...
4,https://openalex.org/W2900756811,https://doi.org/10.1080/20013078.2018.1535750,Minimal information for studies of extracellul...,Minimal information for studies of extracellul...,2018,2018-11-23,{'openalex': 'https://openalex.org/W2900756811...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,"[https://openalex.org/W1506167085, https://ope...","[https://openalex.org/W4393863426, https://ope...","{'ABSTRACT': [0], 'The': [1, 73, 189], 'last':...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 1284}, {'yea...",2024-11-14T12:24:47.770630,2018-11-29,pdf,True,"[[{'raw_affiliation_string': 'Institut Curie, ..."


In [ ]:
argentina_works['raw_affiliation_string'] = argentina_works['affiliations'].apply(lambda x: [d.get('raw_affiliation_string') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
argentina_works['institution_ids']        = argentina_works['affiliations'].apply(lambda x: [d.get('institution_ids') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
argentina_works=argentina_works.explode('institution_ids')
argentina_works.head(2)

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,fulltext_origin,is_authors_truncated,affiliations,raw_affiliation_string,institution_ids
0,https://openalex.org/W2746485780,https://doi.org/10.1038/387253a0,The value of the world's ecosystem services an...,The value of the world's ecosystem services an...,1997,1997-05-01,{'openalex': 'https://openalex.org/W2746485780...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 1163}, {'yea...",2024-11-17T19:37:30.924391,2017-08-31,NaN,NaN,[[{'raw_affiliation_string': 'Center for Envir...,[Center for Environmental and Estuarine Studie...,[https://openalex.org/I149521062]
0,https://openalex.org/W2746485780,https://doi.org/10.1038/387253a0,The value of the world's ecosystem services an...,The value of the world's ecosystem services an...,1997,1997-05-01,{'openalex': 'https://openalex.org/W2746485780...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 1163}, {'yea...",2024-11-17T19:37:30.924391,2017-08-31,NaN,NaN,[[{'raw_affiliation_string': 'Center for Envir...,[Center for Environmental and Estuarine Studie...,[https://openalex.org/I12834331]


In [ ]:
argentina_works=argentina_works[['id','institution_ids','raw_affiliation_string']]
print(argentina_works.head(2))


                                 id                    institution_ids  \
0  https://openalex.org/W2746485780  [https://openalex.org/I149521062]   
0  https://openalex.org/W2746485780   [https://openalex.org/I12834331]   

                              raw_affiliation_string  
0  [Center for Environmental and Estuarine Studie...  
0  [Center for Environmental and Estuarine Studie...  


In [ ]:
# Ensure that 'institution_ids' and 'raw_affiliation_string' have the same number of elements for each row.

# Option 1: Zip the two columns, filter out unequal length entries, then explode

# Create a new column with zipped values
argentina_works['zipped_data'] = argentina_works.apply(
    lambda row: list(zip(row['institution_ids'], row['raw_affiliation_string']))
    if len(row['institution_ids']) == len(row['raw_affiliation_string']) else [], axis=1
)
# Filter out rows with empty zipped_data (unequal length entries)
argentina_works = argentina_works[argentina_works['zipped_data'].apply(len) > 0]

# Explode the zipped column
argentina_works1 = argentina_works.explode('zipped_data')

# Split the zipped data back into original columns
argentina_works1[['institution_ids', 'raw_affiliation_string']] = pd.DataFrame(
    argentina_works1['zipped_data'].tolist(), index=argentina_works1.index
)


In [ ]:
argentina_works1.head(2)

,id,institution_ids,raw_affiliation_string,zipped_data
68,https://openalex.org/W2138820253,https://openalex.org/I4210147873,Instituto de Biología Molecular y Celular de R...,"(https://openalex.org/I4210147873, Instituto d..."
68,https://openalex.org/W2138820253,https://openalex.org/I4141589,Instituto de Biología Molecular y Celular de R...,"(https://openalex.org/I4141589, Instituto de B..."


In [ ]:
argentina_works1['works_id'] = argentina_works1['id'].apply(lambda x: x.split("/")[-1])
#argentina_works1['original_affiliation']=argentina_works1['raw_affiliation_string'].apply(lambda x: x.split("']")[0])
argentina_works1['institution_ids'] = argentina_works1['institution_ids'].apply(lambda x: x.split("/")[-1])
argentina_works1['institution_ids'] =argentina_works1['institution_ids'] .apply(lambda x: x.split("I")[-1])
argentina_works1['affiliation_id']=argentina_works1['institution_ids']
argentina_works1['original_affiliation']=argentina_works1['raw_affiliation_string']

print(argentina_works1.shape)
affiliations=argentina_works1
print(affiliations.head(2))

(1803, 7)
                                  id institution_ids  \
68  https://openalex.org/W2138820253      4210147873   
68  https://openalex.org/W2138820253         4141589   

                               raw_affiliation_string  \
68  Instituto de Biología Molecular y Celular de R...   
68  Instituto de Biología Molecular y Celular de R...   

                                          zipped_data     works_id  \
68  (https://openalex.org/I4210147873, Instituto d...  W2138820253   
68  (https://openalex.org/I4141589, Instituto de B...  W2138820253   

   affiliation_id                               original_affiliation  
68     4210147873  Instituto de Biología Molecular y Celular de R...  
68        4141589  Instituto de Biología Molecular y Celular de R...  


In [ ]:
##agregado para armar dataset de prueba en testeo
print(affiliations.shape)

affiliation_muestra_para_testeo=affiliations.sample(frac=0.1)
affiliation_muestra_para_testeo.shape
affiliation_muestra_para_testeo.head()
affiliation_muestra_para_testeo=affiliation_muestra_para_testeo[['institution_ids','raw_affiliation_string','works_id']]
affiliation_muestra_para_testeo=affiliation_muestra_para_testeo.rename(columns={'institution_ids':'labels', 'works_id': 'paper_id', 'raw_affiliation_string': 'affiliation_string'})
print(affiliation_muestra_para_testeo.head())

affiliation_muestra_para_testeo.to_csv(f"{base_path}V2/002_Model/affiliations_para_testeo.csv")


(1803, 7)
          labels                                 affiliation_string  \
7539   107639228  University of Notre Dame-Mendoza College of Bu...   
6542  4210102755  Centro Internacional de Métodos Computacionale...   
3023  3130805194  Centro de Investigacion Veterinaria de Tandil,...   
6898   151201029  NCAR, P.O. Box 3000, Boulder, Colorado 80307-3...   
1430   189075073  Institute of Materials Science and Technology ...   

         paper_id  
7539  W2137582495  
6542  W2002813019  
3023  W3026900421  
6898  W1964422437  
1430  W2052732316  


In [ ]:
affiliation_muestra_para_testeo.shape

(180, 3)

In [ ]:
##nos quedamos con estas observaciones para affiliations
affiliations1=affiliations-affiliation_muestra_para_testeo

In [ ]:
affiliations1.shape

(2030, 10)

In [ ]:
import os
os.chdir(f"{base_path}V2/002_Model/")

# Now you can safely write the parquet file
affiliations1.to_parquet('affiliations.parquet')

# Assuming 'spark' is your SparkSession object
affiliations = spark.read.parquet(f"{base_path}V2/002_Model/affiliations.parquet")


In [ ]:
print(affiliations.head())

Row(affiliation_id=None, affiliation_string=None, id=None, institution_ids=None, labels=None, original_affiliation=None, paper_id=None, raw_affiliation_string=None, works_id=None, zipped_data=None, __index_level_0__=68)


#### Getting ROR aff strings

In [ ]:
dedup_affs = affiliations.select(F.trim(F.col('original_affiliation')).alias('original_affiliation'), 'affiliation_id')\
.filter(F.col('original_affiliation').isNotNull())\
.filter(F.col('original_affiliation')!='')\
.withColumn('aff_len', F.length(F.col('original_affiliation')))\
.filter(F.col('aff_len')>2)\
.groupby(['original_affiliation','affiliation_id']) \
.agg(F.count(F.col('affiliation_id')).alias('aff_string_counts'))

In [ ]:
dedup_affs.cache().count()

0

In [ ]:
dedup_affs.head(n=3)

[]

In [ ]:
ror_data = spark.read.parquet(f"{base_path}V2/001_Exploration/ror_strings.parquet") \
.select('original_affiliation','affiliation_id')

In [ ]:
ror_data.cache().count()

45963

In [ ]:
ror_data.head(n=2)

[Row(original_affiliation='National University of the Littoral, Santa Fe, Argentina', affiliation_id='3121987637'),
 Row(original_affiliation='National University of the Littoral, Argentina', affiliation_id='3121987637')]

### Gathering training data

Since we are looking at all institutions, we need to up-sample the institutions that don't have many affiliation strings and down-sample the institutions that have large numbers of strings. There was a balance here that needed to be acheived. The more samples that are taken for each institution, the more overall training data we will have and the longer our model will take to train.

However, more samples also means more ways of an institution showing up in an affiliation string.

The number of samples was set to 50 as it was determined this was a good optimization point based on affiliation string count distribution and time it would take to train the model.

However, unlike in V1 where we tried to keep all institutions at 50, for V2 we gave additional samples for institutions with more strings available. Specifically, we allowed those institutions to have up to 25 additional strings, for a total of 75.

In [ ]:
num_samples_to_get = 50

In [ ]:
w1 = Window.partitionBy('affiliation_id')

filled_affiliations = dedup_affs \
    .join(ror_data.select('affiliation_id'), how='inner', on='affiliation_id') \
    .select('original_affiliation','affiliation_id') \
    .union(ror_data.select('original_affiliation','affiliation_id')) \
    .filter(~F.col('affiliation_id').isNull()) \
    .dropDuplicates() \
    .withColumn('random_prob', F.rand(seed=20)) \
    .withColumn('id_count', F.count(F.col('affiliation_id')).over(w1)) \
    .withColumn('scaled_count', F.lit(1)-((F.col('id_count') - F.lit(num_samples_to_get))/(F.lit(3500000) - F.lit(num_samples_to_get)))) \
    .withColumn('final_prob', F.col('random_prob')*F.col('scaled_count'))

In [ ]:
filled_affiliations.select('affiliation_id').distinct().count()


331

In [ ]:
filled_affiliations.head(n=2)

[Row(original_affiliation='Dep. of Chemical and Biomolecular Engineering University of Mendoza', affiliation_id='100084298', random_prob=0.07591612310617579, id_count=147, scaled_count=0.9999722853183617, final_prob=0.0759140191149927),
 Row(original_affiliation='University of Mendoza, Argentina', affiliation_id='100084298', random_prob=0.47680951584976883, id_count=147, scaled_count=0.9999722853183617, final_prob=0.47679630122583494)]

In [ ]:
less_than = filled_affiliations.dropDuplicates(subset=['affiliation_id']).filter(F.col('id_count') < num_samples_to_get).toPandas()

less_than['affiliation_id'] = less_than['affiliation_id'].astype('str')
less_than['affiliation_id'] = less_than['affiliation_id'].apply(lambda x: x.split("/")[-1])

print(less_than.shape)
less_than.head(n=2)


(159, 6)


,original_affiliation,affiliation_id,random_prob,id_count,scaled_count,final_prob
0,IGN,1333839041,0.849786,26,1.000007,0.849791
1,Academia Nacional de Medicina Buenos Aires,14429277,0.777215,26,1.000007,0.777221


In [ ]:
temp_df_list = []
for aff_id in less_than['affiliation_id'].unique():
    temp_df = less_than[less_than['affiliation_id']==aff_id].copy()
    help_df = temp_df.sample(num_samples_to_get - temp_df.shape[0], replace=True)
    temp_df_list.append(pd.concat([temp_df, help_df], axis=0))
less_than_df = pd.concat(temp_df_list, axis=0)

In [ ]:
# only install fsspec and s3fs
less_than_df[['original_affiliation', 'affiliation_id']].to_parquet(f"{iteration_save_path}lower_than_{num_samples_to_get}.parquet")
w1 = Window.partitionBy('affiliation_id').orderBy('random_prob')

more_than = filled_affiliations.filter(F.col('id_count') >= num_samples_to_get) \
.withColumn('row_number', F.row_number().over(w1)) \
.filter(F.col('row_number') <= num_samples_to_get+25)

more_than.cache().count()


more_than=more_than.toPandas()

more_than['affiliation_id'] = more_than['affiliation_id'].astype('str')
more_than['affiliation_id'] = more_than['affiliation_id'].apply(lambda x: x.split("/")[-1])
more_than[['original_affiliation', 'affiliation_id']].to_parquet(f"{iteration_save_path}more_than_{num_samples_to_get}.parquet")


#more_than.select('original_affiliation', 'affiliation_id') \
#.coalesce(1).write.mode('overwrite').parquet(f"{iteration_save_path}more_than_{num_samples_to_get}")

In [ ]:
less_than.sample(5)

,original_affiliation,affiliation_id,random_prob,id_count,scaled_count,final_prob
152,Sociedad Latinoamericana de Medicina del Viajero,4210164372,0.224921,39,1.000003,0.224922
50,"Ministerio de Produccion, Ciencia y Tecnologia...",4210109749,0.210542,13,1.000011,0.210544
81,CCT Bahia Blanca Argentina,4210125789,0.099555,39,1.000003,0.099555
79,Wikimedia Argentina Buenos Aires Buenos Aires ...,4210125659,0.674105,13,1.000011,0.674112
31,"IBMPAS, San Antonio Oeste, Rio Negro, Argentina",4210100874,0.792740,26,1.000007,0.792745


In [ ]:
less_than_df.shape

(7950, 6)

In [ ]:
more_than.head()

,original_affiliation,affiliation_id,random_prob,id_count,scaled_count,final_prob,row_number
0,Dep of Materials Science and Engineering Argen...,4210131417,0.005135,142,0.999974,0.005135,1
1,Argentine Atlantis University Dept. of English...,4210131417,0.005422,142,0.999974,0.005422,2
2,Dept. of Materials Science and Engineering Arg...,4210131417,0.007397,142,0.999974,0.007397,3
3,Argentine Atlantis University Dept of Applied ...,4210131417,0.015353,142,0.999974,0.015353,4
4,Argentine Atlantis University College of Scien...,4210131417,0.027659,142,0.999974,0.027658,5


In [ ]:
more_than.shape

(12659, 7)

In [ ]:
!pip install colab-convert
!colab-convert /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/001_get_training_data_SPARK.ipynb /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/001_get_training_data_SPARK.py

  Preparing metadata (setup.py) ... done
  Created wheel for colab-convert: filename=colab_convert-2.0.5-py3-none-any.whl size=19207 sha256=799d82f75a150d7c1fe2437ab8ff1e5dc7c450995195c05fc49307952ef4a5fb
  Stored in directory: /root/.cache/pip/wheels/a7/9f/0a/70f4be5eeba4a3fca9d7bcf68d5a4e97edb9f22be449cc1e8c
Successfully built colab-convert

---------------------------
----------[START]----------
---------------------------
[OK]   en_US Detected as a supported Language
[OK]   convert magic commands
[OK]   commenting out unsupported magic commands
[OK]   keeping new imports made by converter
[NOT]  showing outputs from converter

convert /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/001_get_training_data_SPARK.ipynb to /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/001_get_training_data_SPARK.py
finished
conversion took 1.614939 seconds

log file created:
/content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/cc-output